In [29]:
import pandas as pd
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import synonyms

torch.manual_seed(1)

[jieba] default dict file path ..\data\vocab.txt
[jieba] default dict file path ..\data\vocab.txt
[jieba] load default dict ..\data\vocab.txt ...
[jieba] load default dict ..\data\vocab.txt ...
>> Synonyms load wordseg dict [D:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\vocab.txt] ... 
>> Synonyms on loading stopwords [D:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\stopwords.txt] ...
[Synonyms] on loading vectors [D:\anaconda\envs\pytorch\lib\site-packages\synonyms\data\words.vector.gz] ...


## BILSTM-CRF模型

### 模型构建

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)

    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))


class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)

        return score, tag_seq

In [3]:
df1 = pd.read_csv('data/游记攻略标注后的数据.csv')
df2 = pd.read_csv('data/酒店评论标注后的数据.csv')
df3 = pd.read_csv('data/景区标注后的数据.csv')
df4 = pd.read_csv('data/餐饮标注后的数据.csv')

In [4]:
print(df2['评论内容'].sample(20))

426                              酒店新开的，还可以！ 我的担保金怎么还没退回来？
28                                                  性价比不错
591                      感觉好有时尚感，没想到在这么小的空间可以享受到整套的智能化设备。
206                                          干净，卫生，服务态度好！
386     爸妈开车路过，对酒店很满意，房间很大，停车方便，有早餐，酒店还有88元自助晚餐，总之符合两位...
1028                   是我出行住得性价比最高的酒店，空调不会对床吹，有电梯，周围吃的很多。
464     酒店就在油城路，地理位置好，酒店停车也方便，还是刚刚开业，装修高大上，房间设施棒棒的。附近还...
714                                              挺好的，性价比高
30      酒店开窗即可见大海，下楼几步就是中国第一滩，停车方便，周边海鲜大排档很多，价格相对实惠便宜，...
364     酒店设有地下和地面停车场，停车方便。酒店服务员服务态度很好，客房干净整洁设施齐全，二三楼有茶...
690                    夜班的服务员特别好，前台和保安特别贴心，下次还会再来，不错的一次体验
1025                                          服务态度很好，酒店舒服
1083                       住了五天，行  交通：方便 卫生：干净 环境：舒服 服务：好
452                                        环境很不错 服务员态度也很好
39                                                  物美价廉！
692     有特别的开床服务，房间小摆件的设计也很有特色。性价比不错，下次还要入住环境优美，地理位置好，...
819                                             服务态度很好很满意
367           

### 划分训练集与测试集

In [5]:
df1.loc[:, ['正文', '实体编码']]

,正文,实体编码
0,1、茂名放鸡岛放鸡岛（原名湾舟岛，又名汾洲岛）是位于广东省茂名市电白区博贺镇东南14.5公里...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,【茂名在哪】我的大茂名位于广东西部，简称粤西，很多人都不知道茂名是哪，所以我介绍的时候一般会...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2,行程总览【地点】茂名【时间】3天（实际游玩时间约2天半）【交通】大巴车、高铁、公交车、网约车...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3,前言几分钟真的吃不遍茂名，题目只是一个噱头🔥🔥这是我第一次写家乡的游记很多店我觉得并不是很好...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
4,茂名原本相对于广州而言，交通稍差，不适合周末二日游。随着江湛铁路的开通，交通已经不成问题。本...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
...,...,...
289,🏨茂名御水古温泉度假酒店广东茂名宝藏小众度假地～🌿环境清幽，景色优美～🎞出片率超高～还不快点...,OOOBIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
290,🉑️骨灰级御水古温泉3日2夜攻略🉑️御水古温泉&amp;浪漫海岸度假区✅帮你省时又省心✅给你...,OOOOOBIIIIOOOOOOOOBIIIIOOOOOOOOOOOOOOOOOOOOOOO...
291,2021年11月25日起了一个大早，6:30就在家门口等候网约顺风车。晨曦初露，云彩丰富。眼...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
292,走马：路上爷爷是一个很重男轻女的传统广东人，但在看到自己曾孙还没有的情况下，就开始叨叨说他的...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...


In [6]:
df1['评论内容'] = df1['正文']
df_train = pd.concat([df1.loc[:, ['评论内容', '实体编码']][:int(len(df1)*0.7)], df2.loc[:, ['评论内容', '实体编码']][:int(len(df2)*0.7)], df3.loc[:, ['评论内容', '实体编码']][:int(len(df3)*0.7)], df4.loc[:, ['评论内容', '实体编码']][:int(len(df4)*0.7)]]) 
df_train.reset_index(drop=True, inplace=True)
df_train_tr = df_train[:int(len(df_train)*0.7)]
df_train_te = df_train[int(len(df_train)*0.7):]

In [7]:
df1['评论内容'] = df1['正文']
df_test = pd.concat([df1.loc[:, ['评论内容', '实体编码']][int(len(df1)*0.7):], df2.loc[:, ['评论内容', '实体编码']][int(len(df2)*0.7):], df3.loc[:, ['评论内容', '实体编码']][int(len(df3)*0.7):], df4.loc[:, ['评论内容', '实体编码']][int(len(df4)*0.7):]]) 
df_test.reset_index(drop=True, inplace=True)

In [8]:
df_train_tr.shape, df_train_te.shape, df_test.shape

((4690, 2), (2010, 2), (2874, 2))

In [9]:
training_data = [(list(a), list(b)) for a, b in zip(df_train_tr['评论内容'], df_train_tr['实体编码'])]
valing_data = [(list(a), list(b)) for a, b in zip(df_train_te['评论内容'], df_train_te['实体编码'])]
predicting_data = [(list(a), list(b)) for a, b in zip(df_test['评论内容'], df_test['实体编码'])]

In [10]:
training_data[:5]

[(['1',
   '、',
   '茂',
   '名',
   '放',
   '鸡',
   '岛',
   '放',
   '鸡',
   '岛',
   '（',
   '原',
   '名',
   '湾',
   '舟',
   '岛',
   '，',
   '又',
   '名',
   '汾',
   '洲',
   '岛',
   '）',
   '是',
   '位',
   '于',
   '广',
   '东',
   '省',
   '茂',
   '名',
   '市',
   '电',
   '白',
   '区',
   '博',
   '贺',
   '镇',
   '东',
   '南',
   '1',
   '4',
   '.',
   '5',
   '公',
   '里',
   '的',
   '岛',
   '屿',
   '。',
   '岛',
   '上',
   '最',
   '高',
   '点',
   '高',
   '1',
   '2',
   '2',
   '米',
   '。',
   '面',
   '积',
   '1',
   '.',
   '9',
   '平',
   '方',
   '公',
   '里',
   '，',
   '是',
   '茂',
   '名',
   '市',
   '最',
   '大',
   '的',
   '海',
   '岛',
   '。',
   '2',
   '、',
   '惠',
   '州',
   '巽',
   '寮',
   '湾',
   '惠',
   '州',
   '巽',
   '寮',
   '湾',
   '（',
   '读',
   '：',
   'x',
   'ù',
   'n',
   'l',
   'i',
   'á',
   'o',
   'w',
   'ā',
   'n',
   '）',
   '位',
   '于',
   '广',
   '东',
   '省',
   '惠',
   '州',
   '市',
   '，',
   '巽',
   '寮',
   '湾',
   '绵',
   '延',
   '1',
   '1',
   '公',
   '里',


In [11]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
# training_data = [(
#     "the wall street journal reported today that apple corporation made money".split(),
#     "B I I I O O O B I O O".split()
# ), (
#     "georgia tech is a university in georgia".split(),
#     "B I O O O O B".split()
# )]
all_data = training_data + valing_data + predicting_data
word_to_ix = {}
for sentence, tags in all_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

In [12]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[::100][0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[::100][0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data[::100]:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[::100][0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!

(tensor(305.7016), [1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 0])
(tensor(1828.8311), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

### 保存模型

In [13]:
torch.save(model.state_dict(), 'data/BiLSTM_CRF_model.pkl')

### 模型测试

In [14]:
val_codes = []
for s in valing_data:
    with torch.no_grad():
        precheck_sent = prepare_sequence(s[0], word_to_ix)
        print(model(precheck_sent))
        val_codes.append(model(precheck_sent)[1])

(tensor(3693.9426), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

(tensor(1244.7971), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(167.7133), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(164.3931), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(156.9490), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(280.0952), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(153.5938), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

(tensor(3075.9685), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

(tensor(296.8389), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(876.1611), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(348.3343), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(193.0876), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(219.2575), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(118.4627), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(107.8189), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(119.4431), 

(tensor(143.1761), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(202.7967), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(23.1512), [2, 2, 2])
(tensor(400.6410), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(100.4865), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(28.3427), [2, 2, 2, 2])
(tensor(211.8362), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(286.7523), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(302.2687), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(268.1053), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(1111.0349), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

...

(tensor(826.9877), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(94.1084), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(90.6133), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(291.6635), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0])
(tensor(17.0387), [2, 2])
(tensor(33.7859), [2, 2, 2, 2])
(tensor(621.2355), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(933.7485), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

(tensor(166.7066), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(179.7740), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(78.8797), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(239.9925), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(260.1693), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(85.6540), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(402.2487), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(693.0583), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

(tensor(1472.0651), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(178.4665), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(841.1406), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(2

(tensor(968.8589), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(118.2329), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(870.0455), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(138.4891), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(16.3240), [2, 2])
(tensor(115.9610), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tens

(tensor(49.6297), [2, 2, 2, 2, 2, 2, 2])
(tensor(35.5279), [2, 2, 2, 2])
(tensor(455.6013), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(831.7637), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(173.7710), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(124.2905), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(127.8767), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(97.3080), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(63.6593), [2, 2, 2, 2, 2, 2, 2, 2])
(tensor(120.9916), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [15]:
df_train_te['预测编码'] = val_codes
df_train_te.head()

D:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,评论内容,实体编码,预测编码
4690,今天我是店里面一开门就进去了，当时一点菜品都没有。我点的这份黑胡椒牛扒真的是我这辈子吃过口感...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4691,#套餐：港式奶茶系列 3 选 1丝袜奶茶口味不错，不过我还是喜欢招牌手打柠檬茶，不怎么爱喝奶...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4692,服务差，不知道老板请这些员工餐厅会不会倒闭，今天的相聚时光已经不是以前的相聚时光了，虽说客寻...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4693,这次就餐整体感觉很赞，比较满意的方面有：口味、环境。,OOOOOOOOOOOOOOOOOOOOOOOOOO,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4694,这个雪媚娘真的好差 榴莲里面没有榴莲 连奥利奥里面都没有奥利奥 只有一块布丁 还有一块蛋糕胚...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


### 模型预测

In [16]:
pre_codes = []
for s in predicting_data:
    with torch.no_grad():
        precheck_sent = prepare_sequence(s[0], word_to_ix)
        print(model(precheck_sent))
        pre_codes.append(model(precheck_sent)[1])

(tensor(18455.4785), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

(tensor(20841.0801), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

(tensor(22512.7148), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

(tensor(21246.4062), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

(tensor(4175.1958), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

...

(tensor(847.3068), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(323.8054), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(24.7749), [2, 2, 2])
(tensor(1042.4131), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(302.3363), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

(tensor(190.3494), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(114.6082), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(303.1472), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(475.5489), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(60.9516), [2, 2, 2, 2, 2, 2, 2, 2])
(tensor(174.5894), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(27.9182), [2, 2, 2, 2])
(tensor(119.6273), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(126.2465), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(259.4891), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(817.4376), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

(tensor(218.2282), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(216.4078), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(856.9817), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(185.6247), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(154.9999), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(58.6812), [2, 2, 2, 2, 2, 2, 2, 2])
(tensor(74.9963), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(173.4863), [2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(158.3555), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(te

(tensor(118.3982), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(205.7082), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(224.6803), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(184.5398), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(159.5111), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(184.2043), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(364.2730), [1, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(76.5653), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(24.5078), [2, 2, 2])
(tensor(286.0594), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(274.9028), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

(tensor(118.8244), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(133.3722), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(149.2835), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(197.4309), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(78.8298), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(45.6338), [2, 2, 2, 2, 2, 2, 2])
(tensor(42.7355), [2, 2, 2, 2, 2, 2, 2])
(tensor(103.6434), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(39.4805), [2, 2, 2, 2, 2, 2])
(tensor(44.9447), [2, 2, 2, 2, 2, 2])
(tensor(271.4388), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(311.9625), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(237.4600), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
(tensor(58.0239), [2, 2, 2, 2, 2, 2, 2, 2])
(

In [17]:
df_test['预测编码'] = pre_codes
df_test.head()

,评论内容,实体编码,预测编码
0,2020年10月4日，我陪来访的亲戚打车到虎头山海滨游玩。我印象中虎头山曾是一个美丽的海滨旅...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,博贺湾大桥雄伟，宛若海龙，来张全家福坐上十几人的小艇，在海面以50迈的速度飞驰，如飞鱼一般跃...,BIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,窦州古城坐落于信宜市镇隆镇八坊村在东、西两江夹趋之下，汇合于鉴江而为窦江，因而古称“窦州”古...,BIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,国庆长假约了几个喜欢摄影的朋友去海边拍日出日落，看渔民撒网捕鱼。在海边漫步，看大海波涛汹涌，...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,大海的美是晴天给的，我们的快乐是大海给的！…………………………………………………契机茂名其实...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


### 保存数据

In [18]:
df_train_tr.to_csv('data/实体识别模型训练数据.csv', index=False)
df_train_te.to_csv('data/实体识别模型测试数据.csv', index=False)
df_test.to_csv('data/实体识别模型预测数据.csv', index=False)

## 实体抽取

In [19]:
# 实体识别
def find_entity_code(conts=None, codes=None):
    entitys = []
    for cont, code in zip(conts, codes):
        entity = ''
        w = list(cont)
        c = list(code)
        for i in range(len(w)):
            if c[i] == 'B' or c[i] == 'I':
                entity += w[i]
                # 判断I后面是否新的实体开始，是则添加前一个实体
                try:
                    if c[i+1] == 'B':
                        entitys.append(entity)
                        entity = ''
                except:
                    pass
            else:
                if entity != '':
                    entitys.append(entity)
                entity = ''
    return entitys

In [20]:
# 实体识别
def find_entity_number(conts=None, numls=None):
    entitys = []
    for cont, numl in zip(conts, numls):
        entity = ''
        w = list(cont)
        c = numl
        for i in range(len(w)):
            if c[i] == 0 or c[i] == 1:
                entity += w[i]
                # 判断I后面是否新的实体开始，是则添加前一个实体
                try:
                    if c[i+1] == 0:
                        entitys.append(entity)
                        entity = ''
                except:
                    pass
            else:
                if entity != '':
                    entitys.append(entity)
                entity = ''
    return entitys

In [21]:
tr_orgion_entity = find_entity_code(df_train_tr['评论内容'], df_train_tr['实体编码'])

In [22]:
val_orgion_entity = find_entity_code(df_train_te['评论内容'], df_train_te['实体编码'])
val_pre_entity = find_entity_number(df_train_te['评论内容'], df_train_te['预测编码'])

In [23]:
pre_orgion_entity = find_entity_code(df_test['评论内容'], df_test['实体编码'])
pre_pre_entity = find_entity_number(df_test['评论内容'], df_test['预测编码'])

In [24]:
tr_orgion_entity

['海滩',
 '海滩',
 '早茶',
 '海鲜',
 '生蚝',
 '荔枝',
 '早茶',
 '早茶',
 '早茶',
 '早茶',
 '早茶',
 '蛋挞',
 '凉拌菜',
 '城市',
 '早茶',
 '早茶',
 '早茶',
 '早茶',
 '早茶',
 '早茶',
 '蛋挞',
 '早茶',
 '博物馆',
 '露天矿生态公园',
 '博物馆',
 '红树林',
 '小东江',
 '第一滩',
 '御水古温泉',
 '城市',
 '酱汁',
 '酱汁',
 '博物馆',
 '博物馆',
 '面包',
 '博物馆',
 '博物馆',
 '博物馆',
 '博物馆',
 '博物馆',
 '城市',
 '博物馆',
 '博物馆',
 '博物馆',
 '骑楼',
 '城市',
 '城市',
 '海鲜',
 '城市',
 '城市',
 '小东江',
 '沙滩',
 '博物馆',
 '海滩',
 '海滩',
 '海滩',
 '海鲜',
 '海鲜',
 '海鲜',
 '荔枝',
 '露天矿',
 '海陵岛',
 '温德姆酒店',
 '温德姆酒店',
 '温德姆酒店',
 '温德姆酒店',
 '温德姆酒店',
 '牛排',
 '海鲜',
 '甜点',
 '浪漫海岸国际旅游度假区',
 '浪漫海岸国际旅游度假区',
 '浪漫海岸国际旅游度假区',
 '浪漫海岸国际旅游度假区',
 '浪漫海岸国际旅游度假区',
 '海陵岛',
 '温德姆酒店',
 '海陵岛',
 '海陵岛',
 '海陵岛',
 '海鲜',
 '沙滩',
 '海鲜',
 '海滩',
 '海鲜',
 '沙滩',
 '海滩',
 '沙滩',
 '沙滩',
 '水东湾',
 '凤凰酒店',
 '海陵岛',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '凤凰酒店',
 '红树林',
 '海鲜',
 '红荔阁',
 '荔枝',
 '

In [35]:
extend_entity = set()
for s in pre_pre_entity:
    for e in all_entity:
        similar = synonyms.compare(s, e, seg=False)
        print(similar)
        if similar > 0.03:
            extend_entity.add(s)
        break

[Synonyms] not exist in w2v model: 海滨旅
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 海滨旅
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 海滨旅
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 滩旅
[Synonyms] not exist in w2v model: 文化广场
0.001
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 海滨旅
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 海滨旅
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 滩旅
[Synonyms] not exist in w2v model: 文化广场
0.001
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.

D:\anaconda\envs\pytorch\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))


0.055
[Synonyms] not exist in w2v model: 哭R
[Synonyms] not exist in w2v model: 文化广场
0.032
[Synonyms] not exist in w2v model: 近海滩
[Synonyms] not exist in w2v model: 文化广场
0.01
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 海日旅
[Synonyms] not exist in w2v model: 文化广场
0.026
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 文化广场
0.024
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 文化广场
0.001
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 文化广场
0.05
[Synonyms] not exist in w2v model: 文化广场
0.055
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms] not exist in w2v model: 文化广场
0.013
[Synonyms]

In [34]:
extend_entity

{'冲冲冲冲',
 '划算外',
 '化旅',
 '外辅',
 '尽量把那',
 '往各镇',
 '德基鸡',
 '油化',
 '油油',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅游度假',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅',
 '浪漫海岸国际旅游度假区集旅',
 '海国际旅',
 '海滩',
 '海滩涂',
 '海滩漫',
 '海银滩',
 '海鲜数',
 '温德姆',
 '温德姆浪漫海岸',
 '温德姆酒店',
 '温油炸',
 '滩涂',
 '纳酒店',
 '量古',
 '际旅游度假',
 '际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅',
 '际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区',
 '鲜滚'}

In [25]:
all_entity = set(tr_orgion_entity + val_orgion_entity + pre_orgion_entity)

In [28]:
pre_pre_entity

['海滨旅',
 '海滩',
 '海滩',
 '沙滩',
 '海滨旅',
 '海滨旅',
 '滩旅',
 '海滩',
 '沙滩',
 '海滨旅',
 '海滨旅',
 '海滩',
 '滩旅',
 '沙滩',
 '海滩',
 '沙滩',
 '沙滩',
 '沙滩',
 '海滩',
 '海滩',
 '海滩',
 '沙滩',
 '海滩',
 '海滩',
 '海滩',
 '海滩',
 '海滩',
 '海滩',
 '沙滩',
 '海滩',
 '海滩',
 '最近',
 '海滩',
 '海鲜',
 '沙滩',
 '沙滩',
 '誉酒店',
 '沙滩',
 '海滩',
 '容纳近',
 '海滩',
 '海滩',
 '沙滩',
 '海鲜',
 '滩这',
 '海鲜',
 '海鲜',
 '滩这',
 '海滩',
 '海滩',
 '海滩',
 '店)',
 '芸芸',
 '滩旅',
 '沙滩',
 '海旅',
 '海旅',
 '海旅',
 '海旅',
 '滩旅',
 '滩景区',
 '海滩',
 '海滩',
 '银滩',
 '海滨旅',
 '海滩',
 '海滩',
 '海鲜',
 '海滩',
 '最近',
 '沙滩',
 '海滩漫',
 '沙滩',
 '沙滩',
 '沙滩',
 '贴近',
 '海滩',
 '沙滩',
 '凑近',
 '油品',
 '油化',
 '品油',
 '海镇',
 '海旅',
 '滩涂',
 '难滩',
 '海滩',
 '温德姆酒店',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅',
 '海鲜',
 '浪漫海岸国际旅',
 '海滩',
 '沙滩',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假',
 '海滩',
 '浪漫海岸国际旅游度假区浪漫海岸国际旅',
 '际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区浪漫海岸国际旅游度假区',
 '海鲜',
 '海海鲜',
 '海鲜',
 '海鲜',
 '最近',
 '油化',
 '最近',
 '王殿',
 '守估算',
 '园油',
 '油脂',
 '旅店',
 '古俚',
 '古俚',
 '滩涂浅滩',
 '滩涂',
 '沙滩',
 '浪漫海岸国际旅',
 '浪漫海岸温德姆酒店',
 '纳酒店',
 '

In [26]:
len(all_entity)

157

In [27]:
df_entity = pd.DataFrame({})
df_entity['实体'] = list(all_entity)
df_entity.to_csv('data/所有实体.csv', index=False)